In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:



# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales_view")



In [5]:

# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:

# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 9.012222290039062e-05 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
df.cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
is_cached = df.is_cached
print("Is home_sales_view cached?", is_cached)

Is home_sales_view cached? True


In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
filtered_result_uncached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE price >= 350000
    GROUP BY view
""")
filtered_result_uncached.show()

print("Runtime is --- %s seconds ---" % (time.time() - start_time))
print("Cashed data is less RunTime than Uncashed data")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Runtime is --- 3.090341091156006 seconds ---
Cashed data is less RunTime than Uncashed data


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [13]:

# 11. Read the parquet formatted data.
p_df=spark.read.parquet('home_sales')


In [14]:

# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home_sales')


In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
  SELECT
  view,
  ROUND(AVG(price), 2)
  FROM home_sales
  WHERE price >=350000
  GROUP BY view
  ORDER BY view  DESC
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 1.3598606586456299 seconds ---


In [18]:

# 14. Uncache the home_sales temporary table.
spark.sql('UNCACHE TABLE home_sales')


DataFrame[]

In [17]:

# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")



False